In [1]:
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os

In [6]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing/')
wd = '/space/hemera/1/users/cmaffei/hcp_processing/'

Preparing tracks and ROIs for TRACULA priors step

In [ ]:
%%script tcsh
#sourcing freesourfer distribution
source /usr/local/freesurfer/nmr-stable60-env
#setting subjects
set subj_list = (mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023)

In [21]:
%%script tcsh
#extracting freesurfer parcels if used for virtual segmentation
source /usr/local/freesurfer/nmr-stable60-env
foreach subj ($subj_list)
        echo $subj
        set indir = $subj/rot_files
        set input = $subj/rot_files/wmparc2diff.nii.gz
        mri_extract_label $input 2031 $indir/rh_supramarginal.nii.gz
        mri_extract_label $input 4025 $indir/rh_precuneus.nii.gz
        mri_extract_label $input 4008 $indir/rh_inf_parietal.nii.gz
end

-------- freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c --------
Setting up environment for FreeSurfer/FS-FAST (and FSL)
FREESURFER_HOME   /usr/local/freesurfer/stable6_0_0
FSFAST_HOME       /usr/local/freesurfer/stable6_0_0/fsfast
FSF_OUTPUT_FORMAT nii.gz
SUBJECTS_DIR      /usr/local/freesurfer/dev/subjects
MNI_DIR           /usr/local/freesurfer/dev/mni
FSL_DIR           /usr/pubsw/packages/fsl/5.0.10
mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023
mgh_1001
reading volume from mgh_1001/rot_files/wmparc2diff.nii.gz...
extracting label 2031 (ctx_rh_supramarginal)
writing output to mgh_1001/rot_files/rh_supramarginal.nii.gz.
reading volume from mgh_1001/rot_files/wmparc2diff.nii.gz...
extracting label 4025 ()
writing output to mgh_1001/rot_files/rh_precuneus.nii.gz.
reading volume from mgh_1001/rot_files/wmparc2diff.nii.gz...
extracting label 4008 ()
writing output to mgh_1001/rot_files/rh_

stdin: is not a tty


In [119]:
%%script tcsh
#rotating rois  back to oblique and then bringing them into MNI
foreach subj ($subj_list)
        echo $subj
        set indir = $subj/rot_files
        echo 'indir is' $indir
        mkdir $subj/rot_files/tmp
        set tmpdir = $subj/rot_files/tmp
        echo 'tmpdir is' $tmpdir
        set outdir = 1k_tracula/$subj/dlabel/mni
        if (-f $indir/slf_stem.nii) then
        cp $indir/slf_stem.nii $indir/slf1_post.nii.gz
        else
        cp $indir/rh_precuneus.nii.gz $indir/slf1_post.nii.gz
        endif
        $FSL_BIN/fslmaths $indir/rh_supramarginal.nii.gz -thr 1 -bin  $indir/rh_supramarginal.nii.gz
        $FSL_BIN/fslmaths $indir/rh_inf_parietal.nii.gz -thr 1 -bin $indir/rh_inf_parietal.nii.gz
        set invols = ( $indir/ant_slf1.nii $indir/ant_slf2.nii $indir/ant_slf3.nii \
                      $indir/rh_supramarginal.nii.gz $indir/rh_inf_parietal.nii.gz $indir/slf1_post.nii.gz)
        foreach vol ($invols)
        echo $vol
                set ref = 1k_tracula/$subj/dmri/dtifit_FA.nii.gz
                set reg = $indir/trans2obliq.mat
                if $vol == $indir/slf1_post.nii.gz then
                flirt -in $vol -ref $ref -out $tmpdir/$vol:t -applyxfm -init $reg
                else
                flirt -in $vol -ref $ref -out $tmpdir/$vol:t -applyxfm -init $reg -interp nearestneighbour
                endif
        end
        
        foreach vol ($tmpdir/*)
                set ref = $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz
                set reg = 1k_tracula/$subj/dmri/xfms/diff2mni.bbr.mat
                if $vol == $indir/slf1_post.nii.gz then
                flirt -in $vol -ref $ref -out $tmpdir/$vol:t -applyxfm -init $reg
                else
                flirt -in $vol -ref $ref -out $outdir/$vol:t -applyxfm -init $reg -interp nearestneighbour
                endif
        end
                mv $outdir/ant_slf1.nii.gz $outdir/rh.slf1_roi1.bbr.nii.gz
                mv $outdir/ant_slf2.nii.gz $outdir/rh.slf2_roi1.bbr.nii.gz
                mv $outdir/ant_slf3.nii.gz $outdir/rh.slf3_roi1.bbr.nii.gz
                mv $outdir/rh_supramarginal.nii.gz $outdir/rh.slf3_roi2.bbr.nii.gz
                mv $outdir/slf1_post.nii.gz $outdir/rh.slf1_roi2.bbr.nii.gz
                mv $outdir/rh_inf_parietal.nii.gz $outdir/rh.slf2_roi2.bbr.nii.gz                   
end
        

-------- freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c --------
Setting up environment for FreeSurfer/FS-FAST (and FSL)
FREESURFER_HOME   /usr/local/freesurfer/stable6_0_0
FSFAST_HOME       /usr/local/freesurfer/stable6_0_0/fsfast
FSF_OUTPUT_FORMAT nii.gz
SUBJECTS_DIR      /usr/local/freesurfer/dev/subjects
MNI_DIR           /usr/local/freesurfer/dev/mni
FSL_DIR           /usr/pubsw/packages/fsl/5.0.10
mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023
mgh_1001
mgh_1001/rot_files
mgh_1001/rot_files/ant_slf1.nii
mgh_1001/rot_files/ant_slf2.nii
mgh_1001/rot_files/ant_slf3.nii
mgh_1001/rot_files/rh_supramarginal.nii.gz
mgh_1001/rot_files/rh_inf_parietal.nii.gz
mgh_1001/rot_files/slf1_post.nii.gz
mgh_1006
mgh_1006/rot_files
mgh_1006/rot_files/ant_slf1.nii
mgh_1006/rot_files/ant_slf2.nii
mgh_1006/rot_files/ant_slf3.nii
mgh_1006/rot_files/rh_supramarginal.nii.gz
mgh_1006/rot_files/rh_inf_parietal.

stdin: is not a tty
Image Exception : #22 :: ERROR: Could not open image mgh_1001/rot_files/tmp/slf1_r.trk
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
Abort (core dumped)
Image Exception : #22 :: ERROR: Could not open image mgh_1001/rot_files/tmp/slf2_r.trk
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
Abort (core dumped)
Image Exception : #22 :: ERROR: Could not open image mgh_1001/rot_files/tmp/slf3_r.trk
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
Abort (core dumped)
Image Exception : #22 :: ERROR: Could not open image mgh_1006/rot_files/tmp/slf1_r.trk
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
Abort (core dumped)
Image Exception : #22 :: ERROR: Could not open image mgh_1006/rot_files/tmp/slf2_r.trk
terminate called after throwing an instance of 'RBD_COMMON::BaseException'
Abort (core dumped)
Image Exception : #22 :: ERROR: Could not open image mgh_1006/rot_file

In [ ]:
%%script tcsh
#rotating tracts  back to oblique and then bringing them into MNI
foreach subj ($subj_list)
        echo $subj
        set indir = $subj/rot_files
        set tmpdir = $subj/rot_files/tmp
        set outdir = 1k_tracula/$subj/dlabel/mni
        #reROTATE TRACTS
        foreach tract ($indir/slf?_r.trk)
                echo $tract:t
#                 convert_xfm -omat $indir/trans2obliq.mat -inverse $indir/obliq2trans.mat
                set ref =  1k_tracula/$subj/dmri/dtifit_FA.nii.gz
                set reg = $indir/trans2obliq.mat
                set source = $subj/rot_files/fa_res.nii.gz
                track_transform -reg $reg -ref $ref -src $source $tract $tmpdir/$tract:t
                #REGISTER TO MNI
                foreach tract ($tmpdir/*)
                set ref = $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz
                set reg = 1k_tracula/$subj/dmri/xfms/diff2mni.bbr.mat
                set source = 1k_tracula/$subj/dmri/dtifit_FA.nii.gz
                track_transform -reg $reg -ref $ref -src $source $tract $outdir/$tract:t
                end
        end
        mv $outdir/slf1_r.trk $outdir/rh.slf1.bbr.trk
        mv $outdir/slf2_r.trk $outdir/rh.slf2.bbr.trk
        mv $outdir/slf3_r.trk $outdir/rh.slf3.bbr.trk
end

In [ ]:
%%script csh
#leave one out script to runa trac-all prior step
set list_subj = (mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023)
set runs = (1 6 7 8 9 10 11 12 13 15 16 17 19 20 22)

foreach index (`seq $#runs`) 
               echo "$list_subj[$index] is run $runs[$index]"
               grep -vwE "$list_subj[$index]" trainlist.txt > oneout.txt
               sed -i "/.*runlist.*/c set runlist= ($runs[$index])" dmrirc.example
               trac-all -c dmrirc.example -prior
end

In [17]:
%%script tcsh

set tracts = (slf1 slf2 slf3)

#transform tracts mni to nifti
# foreach sub ($subj_list) 
#     set indir = 1k_tracula/$sub/dlabel/mni
#     set outdir = 1k_tracula/fig_hist/
#     foreach tract ($tracts)
#         dmri_trk2trk --indir $indir --in rh.$tract.bbr.trk --outdir $outdir --outvol ${sub}_${tract}_manual.nii.gz --inref $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz --outref $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz
#     end
# end

transform tracula paths mni to mni
foreach sub ($subj_list) 
        set indir = 1k_tracula/tmp.$sub/
        set outdir = 1k_tracula/fig_hist/
        set ref = $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz
        set reg = 1k_tracula/$sub/dmri/xfms/diff2mni.bbr.mat
        foreach tract ($tracts)
#         flirt -in $indir/rh.${tract}_avg14_mni_bbr_thr.005.nii.gz -ref $ref -out $outdir/${sub}_${tract}_tracula_005.nii.gz -applyxfm -init $reg
#         flirt -in $indir/rh.${tract}_avg14_mni_bbr_unthr.nii.gz -ref $ref -out $outdir/${sub}_${tract}_tracula_unthr.nii.gz -applyxfm -init $reg
        flirt -in $indir/rh.${tract}_avg14_mni_bbr_thr.05.nii.gz -ref $ref -out $outdir/${sub}_${tract}_tracula_05.nii.gz -applyxfm -init $reg
    end
end

#transform tracula paths mni to mni
# foreach sub ($subj_list) 
#         set indir = 1k_tracula/tmp.$sub/
#         set outdir = 1k_tracula/fig_hist/
#         set ref = $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz
#         set reg = 1k_tracula/$sub/dmri/xfms/diff2mni.bbr.mat
#         foreach tract ($tracts)
#         flirt -in $indir/${tract}_r_tract.nii.gz -ref $ref -out $outdir/${sub}_${tract}_autoptx.nii.gz -applyxfm -init $reg
#     end
# end

-------- freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c --------
Setting up environment for FreeSurfer/FS-FAST (and FSL)
FREESURFER_HOME   /usr/local/freesurfer/stable6_0_0
FSFAST_HOME       /usr/local/freesurfer/stable6_0_0/fsfast
FSF_OUTPUT_FORMAT nii.gz
SUBJECTS_DIR      /usr/local/freesurfer/stable6_0_0/subjects
MNI_DIR           /usr/local/freesurfer/stable6_0_0/mni
FSL_DIR           /usr/pubsw/packages/fsl/current
mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023


stdin: is not a tty
transform: Command not found.


In [120]:
%%script tcsh
#creating the SLF ROIs for auto-ptx
cd /space/hemera/1/users/cmaffei/hcp_processing/1k_tracula/
set slist = (mgh_1001 mgh_1006 mgh_1007 mgh_1008 mgh_1009 mgh_1010 mgh_1011 mgh_1012 mgh_1013 mgh_1015 mgh_1016 mgh_1017 mgh_1019 mgh_1021 mgh_1023)
 mkdir 1k_auto_ptx/rois_mni
foreach roi ($slist[1]/dlabel/mni/rh.slf?_roi?.bbr.nii.gz)
            echo $roi:t
            set outdir = ../1k_auto_ptx/rois_mni
            set path = dlabel/mni
            if $roi == $slist[1]/dlabel/mni/rh.slf1_roi2.bbr.nii.gz then
            $FSL_BIN/fslmaths $slist[4]/$path/$roi:t -add $slist[5]/$path/$roi:t -add $slist[6]/$path/$roi:t -add $slist[7]/$path/$roi:t -add $slist[8]/$path/$roi:t -add $slist[9]/$path/$roi:t -add $slist[10]/$path/$roi:t -add $slist[11]/$path/$roi:t -add $slist[12]/$path/$roi:t -add $slist[13]/$path/$roi:t -add $slist[14]/$path/$roi:t -add $slist[15]/$path/$roi:t $outdir/$roi:t
            else
            $FSL_BIN/fslmaths $slist[1]/$path/$roi:t -add $slist[2]/$path/$roi:t -add $slist[3]/$path/$roi:t -add $slist[4]/$path/$roi:t -add $slist[5]/$path/$roi:t -add $slist[6]/$path/$roi:t -add $slist[7]/$path/$roi:t -add $slist[8]/$path/$roi:t -add $slist[9]/$path/$roi:t -add $slist[10]/$path/$roi:t -add $slist[11]/$path/$roi:t -add $slist[12]/$path/$roi:t -add $slist[13]/$path/$roi:t -add $slist[14]/$path/$roi:t -add $slist[15]/$path/$roi:t $outdir/$roi:t
            endif
            echo "done"
end

mgh_1001
rh.slf1_roi2.bbr.nii.gz
fslmaths mgh_1001/dlabel/mni/rh.slf1_roi2.bbr.nii.gz -add mgh_1006/dlabel/mni/rh.slf1_roi2.bbr.nii.gz ../1k_auto_ptx/rois_mni/rh.slf1_roi2.bbr.nii.gz
done


stdin: is not a tty


In [132]:
%%script tcsh
set indir = 1k_auto_ptx/rois_mni
#thresholding ROIs for voxels present in at least 5 subjects
foreach roi ($indir/*)
    if $roi == $indir/rh.slf1_roi2.bbr.nii.gz then 
    echo "nope"
    else 
    echo ${roi:t:r:r}_bin.nii.gz
    $FSL_BIN/fslmaths $roi -thr 5 -bin $indir/${roi:t:r:r}_bin.nii.gz
    endif
end

rh.slf1_roi1.bbr_bin.nii.gz
nope
rh.slf2_roi1.bbr_bin.nii.gz
rh.slf2_roi2.bbr_bin.nii.gz
rh.slf3_roi1.bbr_bin.nii.gz
rh.slf3_roi2.bbr_bin.nii.gz


stdin: is not a tty


In [ ]:
%%script csh
mrtrixenv

#computing Hausdorf distance, jacard, and Dice between manual labels and TRACULA/MULTI-ROIS
foreach sub ($subj_list)
    set pathdir = $sub/dpath
    set tractdir = ../$sub/rot_files/tmp
    set tmpdir = tmp.$sub
    
    mkdir $tmpdir
    # Transform manual labels from .trk to volume
    foreach tract ($tractdir/slf?_r.trk)
        echo $tract
        dmri_trk2trk --indir $tractdir --in $tract:t --outdir $tmpdir --outvol ${tract:t:r}.nii.gz --inref $sub/dmri/dtifit_FA.nii.gz --outref $sub/dmri/dtifit_FA.nii.gz
    end
    
    #Threshold and binarize Tracula posterior distributions
    set thr = .005
    foreach paths ($pathdir/rh.slf?_avg14_mni_bbr)
        set pmax = `fslstats $paths/path.pd.nii.gz -R | awk '{print $2}'`
        set pthr = `echo "($pmax * $thr +1)/1" | bc `
        fslmaths $paths/path.pd.nii.gz -thr $pthr $tmpdir/${paths:t}_thr$thr.nii.gz
        cp $paths/path.pd.nii.gz $tmpdir/${paths:t}_unthr.nii.gz
        mri_binarize --i $paths/path.pd.nii.gz --min $pthr --o $tmpdir/${paths:t}_bin_thr$thr.nii.gz
    end
    
    # Resize and binarize autoptx tracks (auto-ptx outputs paths at 1x1x1)
    set tracts = (slf1 slf2 slf3)
    foreach tract ($tracts)
        set trac_autoptx_unthr = /space/hemera/1/users/cmaffei/hcp_processing/1k_auto_ptx/auto_ptx/tracts/$sub/${tract}_r/tracts/tractsNorm.nii.gz    
        set trac_autoptx = /space/hemera/1/users/cmaffei/hcp_processing/1k_auto_ptx/auto_ptx/visualise/th_0.005/$sub/${tract}_r_tract.nii.gz
        mrresize $trac_autoptx -vox 1.5,1.5,1.5 $tmpdir/${trac_autoptx:t:r:r}.nii.gz -interp nearest
        mrresize $trac_autoptx_unthr -vox 1.5,1.5,1.5 $tmpdir/${tract}_autoptx_unthr.nii.gz -interp nearest
        fslmaths $tmpdir/${tract}_autoptx_unthr.nii.gz -thr 0.000000001 -bin $tmpdir/${tract}_autoptx_unthr_bin.nii.gz
        fslmaths $tmpdir/${trac_autoptx:t:r:r}.nii.gz -thr 0.001 -bin $tmpdir/${trac_autoptx:t:r:r}_bin.nii.gz
    end
    
    # Compute measures. Auto-ptx thresholds at 0.005. We try different threshold for TRACULA and the unthreshold version for autoptx
    set tracts = (slf1 slf2 slf3)
    set thrs = (unthr thr.005 thr.05 thr.1 thr.15 thr.2) #tracula thresholds
    foreach tract ($tracts)
        fslmaths $tmpdir/${tract}_r.nii.gz -thr 1 -bin $tmpdir/${tract}_r_bin.nii.gz
        set true_tract = $tmpdir/${tract}_r_bin.nii.gz #manual_label
        set tract_autoptx = $tmpdir/${tract}_r_tract_bin.nii.gz
        set tract_autoptx_unthr = $tmpdir/${tract}_autoptx_unthr_bin.nii.gz
        mri_hausdorff_dist -b 1 $true_tract $tract_autoptx_unthr $tmpdir/${tract}_accuracy_autoptx_unthr.txt
        mri_compute_overlap -l $tmpdir/${tract}_overlap_autoptx_unthr $true_tract $tract_autoptx_unthr 1
        mri_hausdorff_dist -b 1 $true_tract $tract_autoptx $tmpdir/${tract}_accuracy_autoptx.txt
        mri_compute_overlap -l $tmpdir/${tract}_overlap_autoptx $true_tract $tract_autoptx 1
        foreach thr ($thrs)
            set trac_tract = $tmpdir/rh.${tract}_avg14_mni_bbr_bin_${thr}.nii.gz
            mri_hausdorff_dist -b 1 $true_tract $trac_tract $tmpdir/${tract}_accuracy_$thr.txt
            mri_compute_overlap -l $tmpdir/${tract}_overlap_${thr}.txt $true_tract $trac_tract 1
        end
    end            
end